In [1]:
import sqlite3
import pandas as pd

In [10]:
ontime = pd.read_csv("/Users/levbarbash/LondonProg/Assignment3/2000.csv")
for year in range(2001, 2006):
    name = "/Users/levbarbash/LondonProg/Assignment3/" + str(year) + ".csv"
    ontime = pd.concat([ontime, pd.read_csv(name)])
airports = pd.read_csv("/Users/levbarbash/LondonProg/Assignment3/airports.csv")
carriers = pd.read_csv("/Users/levbarbash/LondonProg/Assignment3/carriers.csv")
planes = pd.read_csv("/Users/levbarbash/LondonProg/Assignment3/plane-data.csv")
conn = sqlite3.connect('/Users/levbarbash/LondonProg/Assignment3/airline2.db')
ontime.to_sql('ontime', con = conn, index = False) 
airports.to_sql('airports', con = conn, index = False) 
carriers.to_sql('carriers', con = conn, index = False)
planes.to_sql('planes', con = conn, index = False)
conn.close()

/var/folders/wm/r88x234s74db86mtqppjmvq00000gn/T/ipykernel_36598/3417453225.py:4: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  ontime = pd.concat([ontime, pd.read_csv(name, encoding='latin-1')])


In [18]:
def execute_sql_query(query):
    conn = sqlite3.connect('/Users/levbarbash/LondonProg/Assignment3/airline2.db')
    q_planes = conn.execute(query).fetchall()
    conn.close()
    return pd.DataFrame(q_planes)

Planes with lowest average departure delay

In [19]:
execute_sql_query('''
SELECT model AS model, AVG(ontime.DepDelay) AS avg_delay
FROM planes JOIN ontime USING(tailnum)
WHERE ontime.Cancelled = 0 AND ontime.Diverted = 0 AND ontime.DepDelay > 0
GROUP BY model
ORDER BY avg_delay''')

,0,1
0,737-2Y5,7.022026
1,737-282,8.433566
2,737-230,10.458647
3,737-282C,11.765766
4,767-324,15.788051
...,...,...
141,ATR 72-212,38.267365
142,ATR-72-212,38.699195
143,717-200,38.737105
144,ERJ 190-100 IGW,41.723958


Cities with the highest number of inbound flights

In [20]:
execute_sql_query('''
SELECT airports.city AS city, COUNT(*) AS total
FROM airports JOIN ontime ON ontime.dest = airports.iata
WHERE ontime.Cancelled = 0
GROUP BY airports.city
ORDER BY total DESC''')

,0,1
0,Chicago,2388365
1,Atlanta,1912370
2,Dallas-Fort Worth,1781833
3,Houston,1304407
4,Los Angeles,1279514
...,...,...
282,Provo,3
283,Cheyenne,2
284,Scottsbluff,1
285,Roswell,1


Сompanies with the highest number of cancelled flights

In [21]:
execute_sql_query('''
SELECT carriers.Description AS carrier, COUNT(*) AS total
FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
WHERE ontime.Cancelled = 1
    AND carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
GROUP BY carriers.Description
ORDER BY total DESC''')

,0,1
0,Delta Air Lines Inc.,107851
1,American Airlines Inc.,105762
2,United Air Lines Inc.,102066


Companies with the highest number of cancelled flights, relative to their number of total flights

In [22]:
execute_sql_query('''
SELECT
    q1.carrier AS carrier, (CAST(q1.numerator AS FLOAT)/ CAST(q2.denominator AS FLOAT)) AS ratio
FROM
(
SELECT carriers.Description AS carrier, COUNT(*) AS numerator
FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
WHERE ontime.Cancelled = 1 AND carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
GROUP BY carriers.Description
) AS q1 JOIN
(
SELECT carriers.Description AS carrier, COUNT(*) AS denominator
FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
WHERE carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
GROUP BY carriers.Description
) AS q2 USING(carrier)
ORDER BY ratio DESC''')

,0,1
0,United Air Lines Inc.,0.027924
1,Delta Air Lines Inc.,0.024082
2,American Airlines Inc.,0.023841
